In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 1. THE DATA: Our raw training text
text = """artificial intelligence is transforming modern society. it is used in healthcare.
machine learning allows systems to improve automatically. data plays a critical role."""
text = text.lower()

# 2. TOKENIZATION: Map characters to numbers so the computer can "read" them
chars = sorted(list(set(text))) # Get unique letters/symbols
char_to_idx = {char: i for i, char in enumerate(chars)}
idx_to_char = {i: char for i, char in enumerate(chars)}

# 3. SEQUENCING: Cut the text into "slices" (Input = 40 chars, Output = the next 1 char)
seq_length = 20
input_data = []
output_data = []

for i in range(0, len(text) - seq_length):
    input_data.append([char_to_idx[c] for c in text[i : i + seq_length]])
    output_data.append(char_to_idx[text[i + seq_length]])

# Reshape data to fit the model [Samples, Time Steps, Features]
X = np.reshape(input_data, (len(input_data), seq_length, 1)) / float(len(chars))
y = tf.keras.utils.to_categorical(output_data)

# 4. THE BRAIN (LSTM): Building the layers
model = Sequential([
    LSTM(128, input_shape=(X.shape[1], X.shape[2])), # Memory layer
    Dense(y.shape[1], activation='softmax')           # Decision layer
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

# 5. TRAINING: The model reads the text 50 times to learn patterns
model.fit(X, y, epochs=50, batch_size=32, verbose=0)

# 6. GENERATION: Start with a "seed" and let the model finish the sentence
seed = "artificial intellig " # Added a space to make it seq_length (20) characters long
generated_text = seed

for i in range(50):
    # Prepare the seed
    x_input = np.reshape([char_to_idx[c] for c in seed[-seq_length:]], (1, seq_length, 1)) / float(len(chars))
    # Predict the next letter
    prediction = model.predict(x_input, verbose=0)
    next_idx = np.argmax(prediction)
    # Add to the text
    seed += idx_to_char[next_idx]
    generated_text += idx_to_char[next_idx]

print(f"Generated Result: {generated_text}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Generated Result: artificial intellig     aaaaaaaaaaaaaaa aaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
